### Capstone Project - Assignment week 3b: Neighborhood Segmentation and Clustering in Toronto

This weeks assignment is about explore and cluster the neighborhoods in Toronto:  
1) Get Data from Wiki importing required libraries to get data from wiki  
2) Clean the data and make a pandas dataframe  
3) Use the Geocoder package to find coordinates (lat/long)

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

1. <a href="#section1">Download, Explore and Clean Dataset</a>
2. <a href="#section2">Get the coordinates of the neighborhoods</a>


</font>
</div>

<a id="section1"></a>
# 1. Download, Explore and Clean Dataset

#### Import the modules

In [1]:
# Import modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import lxml
import csv
from collections import Counter

#### Get the data from the web page using request

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

#### The website data can be view in a pretty format by using beautifulsoup's prettify() method. 

In [3]:
soup = BeautifulSoup(page.content, 'lxml')

#### The first task is to find class ‘wikitable sortable’ in the script. So, first we will extract the data in table tag using find method of bs4 object.

In [4]:
my_table = soup.find('table', class_='wikitable')

In [5]:
links = my_table.findAll('td')
links

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights

In [6]:
tag = soup.a

In [7]:
tag.attrs

{'id': 'top'}

In [8]:
# Let’s get all of the HTML tags from inside the body of the page.
tags = [tag.name for tag in soup.body.find_all(True)]
len(tags)

1895

In [9]:
# Count up how many of each tag type are present.
tag_counts = Counter(tags)
len(tag_counts)

27

In [10]:
tag_counts.most_common(5)

[('td', 901), ('a', 468), ('tr', 295), ('li', 62), ('div', 43)]

#### Declaring Empty Dataframe with three columns

In [11]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
column_names

['PostalCode', 'Borough', 'Neighborhood']

#### Reading Data from tags transform the data into a pandas dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [12]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    
    #print(len(columns), columns)
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
neighborhoods.head()

PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

#### Only process the cells that have an assigned borough. Ignored the cells with a borough that is not assigned.

In [13]:
neighborhoods.shape

(289, 3)

In [14]:
neighborhoods['Borough'].value_counts()['Not assigned']

77

In [15]:
df_neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]
df_neighborhoods.shape

(212, 3)

In [16]:
# Check: 289 (total lenght of list) - 77 (rows with not assigned borough, 
# ignored) = 212, so far so good

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [25]:
df_neighborhoods['Neighborhood'].replace(to_replace="Not assigned", value=df_neighborhoods['Borough'], inplace=True)

In [18]:
df_neighborhoods.head(5)

PostalCode           Borough      Neighborhood
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M5A  Downtown Toronto       Regent Park
6        M6A        North York  Lawrence Heights

In [26]:
df_neigh = pd.DataFrame({'Neighborhood' : df_neighborhoods.groupby(by = ['PostalCode', 
                                                                      'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()

In [27]:
df_neigh.head()

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [28]:
df_neigh.shape

(103, 3)

In [29]:
df_neigh.to_csv("df_neighborhoods.csv")

<a id="section2"></a>
# 2. Get the coordinates of the neighborhoods

#### Reading data from Geospatial_Data file from http://cocl.us/Geospatial_data

In [21]:
import wget
url = "http://cocl.us/Geospatial_data"
filename = wget.download(url)

100% [................................................................................] 2891 / 2891

In [22]:
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")
df_coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

#### Merge two dataframes to add coordinates

In [30]:
geo_result = (pd.merge(df_neigh, df_coordinates, left_on="PostalCode", right_on="Postal Code").
            drop('Postal Code', axis=1))

In [31]:
geo_result.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [32]:
geo_result.shape

(103, 5)